# 数据清洗

In [87]:
import numpy as np
import pandas as pd

### 税收不遵从

In [88]:
deregulation = pd.concat([
    pd.read_excel('RESSET_DEREGULATION_EXT_1.xls', engine='xlrd'),
    pd.read_excel('RESSET_DEREGULATION_EXT_2.xls', engine='xlrd')
], ignore_index=True)
deregulation['year'] = deregulation['信息发布日期_InfoPubDt'].dt.year - 1


In [89]:
# 检查处理原因和处分措施中是否包含关键词
pattern = '|'.join(['税'])
tax_noncomp = deregulation[
    deregulation['涉及法规_InvolStatute'].astype(str).str.contains(pattern, regex=True) # |
    # deregulation['处分措施_Pena'].astype(str).str.contains(pattern, regex=True)
]
tax_noncomp = tax_noncomp.drop_duplicates(subset=['year', '上市公司代码_ComCd'])[['上市公司代码_ComCd','最新公司全称_LComNm', 'year']]
tax_noncomp['noncompliance'] = 1

### 财报指标

In [90]:
finratio = pd.concat([
    pd.read_excel('RESSET_FINRATIO_1.xls', engine='xlrd'),
    pd.read_excel('RESSET_FINRATIO_2.xls', engine='xlrd'),
    pd.read_excel('RESSET_FINRATIO_3.xls', engine='xlrd')
], ignore_index=True)
finratio = finratio.rename(columns={'上市公司代码_Comcd': '上市公司代码_ComCd', 
                                    '最新公司全称_Lcomnm': '最新公司全称_LComNm'})
finratio['year'] = finratio['截止日期_Enddt'].dt.year


### 股权指标

In [91]:
share_type = pd.read_excel('RESSET_MSHLSTANDOS_1.xls', engine='xlrd')
share_type['year'] = share_type['截止日期_EndDt'].dt.year
share_type['股东类别_SHType'] = (share_type['股东类别编码_SHTypeCd'] == 10).astype(int)

share_num = pd.read_excel('RESSET_SHNUMBER_1.xls', engine='xlrd')
share_num['year'] = share_num['截止日期_EndDt'].dt.year

### 合并

In [92]:
data = share_type[['上市公司代码_ComCd', '最新公司全称_LComNm', 'year']]
data = pd.merge(data,
                tax_noncomp[['上市公司代码_ComCd', 'year', 'noncompliance']],
                on=['year', '上市公司代码_ComCd'],
                how='left')
data['noncompliance'] = data['noncompliance'].fillna(0)
data = pd.merge(data, 
                share_type[['year', '上市公司代码_ComCd', '股东类别_SHType']],
                on=['year', '上市公司代码_ComCd'],
                how='left')
data = pd.merge(data, 
                share_num[['year', '上市公司代码_ComCd', '股东总户数(户)_SHNum', '户均持股数(股/户)_AvgHS']],
                on=['year', '上市公司代码_ComCd'],
                how='left')
data = pd.merge(data,
                finratio[[col for col in finratio.columns if col not in ['更新日期_Updt', '观测序号_nobs', '公司代码_CompanyCode', '截止日期_Enddt', '最新公司全称_LComNm']]],
                on=['year', '上市公司代码_ComCd'],
                how='left')

In [93]:
# # 按noncompliance分组，分别统计缺失数量和比例
# grouped = data.groupby('noncompliance')

# # 统计缺失数量
# missing_count = grouped.apply(lambda g: g.isnull().sum()).T
# missing_count.columns = ['缺失数_合规', '缺失数_不合规']

# # 统计缺失比例
# missing_ratio = grouped.apply(lambda g: g.isnull().mean()).T
# missing_ratio.columns = ['缺失率_合规', '缺失率_不合规']

# # 合并为一个表格
# missing_summary = pd.concat([missing_count, missing_ratio], axis=1)

# # 可选：按不合规样本的缺失率降序排序
# missing_summary = missing_summary.sort_values('缺失率_不合规', ascending=False)

# # 显示前若干行
# print(missing_summary.head(10))  # 或使用 missing_summary.to_excel("缺失统计.xlsx")

In [94]:
# 样本、变量清洗
vars_kept = ['上市公司代码_ComCd',
             '最新公司全称_LComNm',
             'year',
             'noncompliance',
             '股东类别_SHType', 
             '股东总户数(户)_SHNum',
             '户均持股数(股/户)_AvgHS',
             '每股收益(元/股)_BasicEPS', 
             '每股经营活动现金流量(元/股)_OpeCFPS',
             '每股现金及现金等价物余额(元/股)_CCEPS', 
             '净利润()_NetPrf', 
             '营业利润/营业总收入()_OpePrTOR', 
             '有形净值债务率(%)_DbTanEquRt', 
             '股东权益/负债合计_EquTotLia', 
             '资产负债率(%)_DbAstRt',
             '产权比率(%)_DbEquRt',
             '利润总额增长率(%)_TotPrfGrRt',
             '营业收入3年复合增长率(%)_OperaInc3GrRt',
             '股东权益周转率(次)_EquRat',
             '经营现金净流量(元)_NOCF', 
             '营业收入增长率(%)_OpeIncmGrRt',
             '每股资本公积金(元/股)_CapSurFdPS',
             '权益乘数(%)_EquMul',
            #  '净资产收益率(扣除，加权)(%)_WROECut',
            #  '流动资产/总资产(%)_CurTotAst',
            #  '销售成本率(%)_SalCostRt',
            #  '销售毛利率(%)_GIncmRt',
            #  '销售期间费用率(%)_PdCostRt',
            #  '非流动资产/总资产(%)_NoncurTotAst',
             ]

data_cleaned = data[vars_kept].dropna()
# data_cleaned = data[[col for col in data.columns if col not in ['利润总额/息税前利润(%)_TPEBITDDP']]].dropna()

### 合规企业欠采样

In [95]:
foo = []
for year, group in data_cleaned.groupby('year'):
    positive = group[group['noncompliance'] == 1]
    negative = group[group['noncompliance'] == 0]
    n_pos = len(positive)
    n_neg = min(len(negative), n_pos * 5)

    negative_sample = negative.sample(n=n_neg, random_state=42)

    foo.append(pd.concat([positive, negative_sample]))

data_sample = pd.concat(foo).reset_index(drop=True)


### 保存数据

In [96]:
kept_cols = [col for col in data_sample.columns if col not in ['上市公司代码_ComCd', '最新公司全称_LComNm', 'year']]
data_sample[kept_cols].to_csv('../data.csv', index=False, encoding='utf-8-sig')